In [1]:
import pandas as pd
import pickle

In [4]:
train_data = pd.read_csv('../data/processed/features_for_models.csv')
test_data = pd.read_csv('../data/processed/test_dataset.csv')

In [5]:
with open('../artifacts/pipeline.pkl', 'rb') as f:
    titanic_survived_model = pickle.load(f)

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Agregar lectura de target con el archivo de configuraciones

In [ ]:
x_features = train_data.drop(labels=['Survived'], axis=1)
y_target = train_data['Survived']

x_features_test = test_data.drop(labels=['Survived'], axis=1)
y_target_test = test_data['Survived']

In [15]:
rf_model = RandomForestClassifier().fit(x_features, y_target)
y_preds = rf_model.predict(x_features_test)
rf_acc = accuracy_score(y_preds, y_target_test)
rf_acc

0.7835820895522388

In [19]:
lr_model = LogisticRegression().fit(x_features, y_target)
y_preds_lr = lr_model.predict(x_features_test)
lr_acc = accuracy_score(y_preds_lr, y_target_test)
lr_acc

0.8022388059701493

In [ ]:
if(rf_acc > lr_acc):
    titanic_survived_model.steps.append(
        ('modelo_random_forest', RandomForestClassifier()))
else:
    titanic_survived_model.steps.append(
        ('modelo_regresion_logistica', LogisticRegression()))

In [ ]:
titanic_survived_model.fit()

In [ ]:
dataset_test = pd.read_csv('../data/raw/test.csv')
dataset_test.drop(labels=['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1, inplace=True)
titanic_survived_model.predict(dataset_test)

NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.